In [14]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

import random

In [4]:
def simulate_bivariate_abpnl(n: int) -> dict: 
    def f1(x: np.array) -> np.array:
        return x**(-1) + 10*x
    def f2(z: np.array) -> np.array:
        return z**3
    
    x = np.random.uniform(0.1, 1.1, n)
    noise = np.random.uniform(0, 5, n)
    
    z = f1(x) + noise
    y = f2(z)
    df = pd.DataFrame({'x1': x, 'x2': y})
    sim_data = {'df': df, 'noise': noise}
    
    return sim_data

In [5]:
def centering(M):
    n = M.shape[0]
    mat_ones = torch.ones((n, n))
    idendity = torch.eye(n)
    H = idendity - mat_ones/n
    
    C = torch.matmul(M, H)
    return C
    
    
def gaussian_grammat(x, sigma2=None):
    xxT = torch.squeeze(torch.matmul(x, x.T))
    x2 = torch.diag(xxT)
    xnorm = x2 - xxT + (x2 - xxT).T
    
    if sigma2 is None:
        sigma2 = torch.median(xnorm[xnorm != 0])
        
    if sigma2 == 0:
        sigma2 += 1e-16
        
    Kx = torch.exp(-xnorm/sigma2)
    
    return Kx
    
def HSIC(x, y):
    gram_x = gaussian_grammat(x)
    gram_y = gaussian_grammat(y)
    
    c = x.shape[0]**2
    hsic = torch.trace(torch.matmul(centering(gram_x), centering(gram_y)))/c
    
    return hsic

In [6]:
x = torch.Tensor(np.random.normal(size=1000).reshape((-1, 1, 1)))
y = torch.Tensor(np.random.normal(size=1000).reshape((-1, 1, 1)))
print(HSIC(x, y))
print(HSIC(x, x))

tensor(0.0002)
tensor(0.1009)


In [25]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        data = self.data[idx, :]
        
        return data[:-1].reshape((-1, 1)), data[-1].reshape((-1, 1))
    

class Network(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, input_dim)
#             nn.LeakyReLU()
        )
        
        self.encode = nn.Sequential(
            nn.Linear(input_dim, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, input_dim)
#             nn.LeakyReLU()
        )
        self.decode = nn.Sequential(
            nn.Linear(input_dim, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, input_dim)
#             nn.LeakyReLU()
        )
        
    def forward(self, x, y):
        g1_x = self.network(x)
        g3_y = self.encode(y)
        y_approx = self.decode(g3_y)
        
        assert y.shape == y_approx.shape
        
        return [g1_x, y_approx, g3_y]
    
def train_model(train_loader, test_loader, num_epochs, input_dim, log_every_batch = 10):
    device = 0 if torch.cuda.is_available() else 'cpu'
    model = Network(input_dim).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999))

    train_loss_avgs = []
    test_loss_avgs = []
    
    min_loss = 10000

    for epoch in range(num_epochs):
        model.train()
        train_loss_trace = []

        for batch, (x, y) in enumerate(train_loader):
            x = x.to(device)
            x = x.float()
            y = y.to(device)
            y = y.float()

            g1_x, y_approx, g3_y = model.forward(x, y)
            noise = g3_y - g1_x

            loss = lamb*F.mse_loss(y_approx, y) + (1-lamb)*HSIC(x, noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_trace.append(loss.detach().item())
            if batch % log_every_batch == 0:
                print(f'Training: epoch {epoch} batch {batch} loss {loss}')

        model.eval()
        test_loss_trace = []
        for batch, (x, y) in enumerate(test_loader):
            x = x.to(device)
            x = x.float()
            y = y.to(device)
            y = y.float()

            g1_x, y_approx, g3_y = model.forward(x, y)
            noise = g3_y - g1_x

            loss = lamb*F.mse_loss(y_approx, y) + (1-lamb)*HSIC(x, noise)

            test_loss_trace.append(loss.detach().item())
            if batch % log_every_batch == 0:
                print(f'Test: epoch {epoch} batch {batch} loss {loss}')

        train_avg = np.mean(train_loss_trace)
        test_avg = np.mean(test_loss_trace)
        
        if test_avg < min_loss:
            min_loss = test_avg

        train_loss_avgs.append(train_avg)
        test_loss_avgs.append(test_avg)
        print(f'epoch {epoch} finished - avarage train loss {train_avg} ',
             f'avarage test loss {test_avg}')
        
    return train_loss_avgs, test_loss_avgs, min_loss

In [26]:
n = 1000

batch_size = 32
lamb = 0.5
num_epochs = 200

num_trials = 9

In [27]:
data = simulate_bivariate_abpnl(n)
df = data['df']
df = (df-df.mean())/df.std()
noise = data['noise']

In [28]:
def get_final_median_loss(df, num_trials):
    rand_seed = np.random.randint(0, 1000000)
    random.seed(rand_seed)
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)

    input_dim = df.shape[1] - 1

    train, test = train_test_split(df, test_size=0.1, random_state=10, shuffle=True)

    train = np.array(train)
    test = np.array(test)

    train = MyDataset(train)
    test = MyDataset(test)
    
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, 
                              num_workers=0, pin_memory=True)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=False,
                             num_workers=0, pin_memory=True)
    
    losses = []
    for trial in range(num_trials):
        train_loss_avgs, test_loss_avgs, min_loss = train_model(train_loader, test_loader, num_epochs, input_dim)
        losses.append(min_loss)
    
    median_loss = np.median(losses)
    return median_loss, losses

In [29]:
# for b, (x, y) in enumerate(train_loader):
#     print(b)
#     print(x.shape)
#     print(y.shape)

In [30]:
median_loss, losses = get_final_median_loss(df, num_trials)

Training: epoch 0 batch 0 loss 0.6096925735473633
Training: epoch 0 batch 10 loss 0.6297626495361328
Training: epoch 0 batch 20 loss 0.3950054943561554
Test: epoch 0 batch 0 loss 0.3975762724876404
epoch 0 finished - avarage train loss 0.5092814030318424  avarage test loss 0.43740154430270195
Training: epoch 1 batch 0 loss 0.6242485642433167
Training: epoch 1 batch 10 loss 0.49413564801216125
Training: epoch 1 batch 20 loss 0.4532685875892639
Test: epoch 1 batch 0 loss 0.39700934290885925
epoch 1 finished - avarage train loss 0.513841894166223  avarage test loss 0.43810373172163963
Training: epoch 2 batch 0 loss 0.9149767756462097
Training: epoch 2 batch 10 loss 0.41105225682258606
Training: epoch 2 batch 20 loss 0.5483582615852356
Test: epoch 2 batch 0 loss 0.32973232865333557
epoch 2 finished - avarage train loss 0.4945511047182412  avarage test loss 0.3701705299317837
Training: epoch 3 batch 0 loss 0.4995259642601013
Training: epoch 3 batch 10 loss 0.3498347997665405
Training: epoch

Training: epoch 28 batch 10 loss 0.013228220865130424
Training: epoch 28 batch 20 loss 0.006070566363632679
Test: epoch 28 batch 0 loss 0.017024841159582138
epoch 28 finished - avarage train loss 0.013882953689658436  avarage test loss 0.018758974270895123
Training: epoch 29 batch 0 loss 0.007323897443711758
Training: epoch 29 batch 10 loss 0.002524087205529213
Training: epoch 29 batch 20 loss 0.008286064490675926
Test: epoch 29 batch 0 loss 0.016069408506155014
epoch 29 finished - avarage train loss 0.007599582100800913  avarage test loss 0.016017678892239928
Training: epoch 30 batch 0 loss 0.0026284032501280308
Training: epoch 30 batch 10 loss 0.0025577035266906023
Training: epoch 30 batch 20 loss 0.004371720366179943
Test: epoch 30 batch 0 loss 0.015329928137362003
epoch 30 finished - avarage train loss 0.007316525383242245  avarage test loss 0.020567144732922316
Training: epoch 31 batch 0 loss 0.007602439261972904
Training: epoch 31 batch 10 loss 0.014409455470740795
Training: epoc

Training: epoch 56 batch 10 loss 0.006797271780669689
Training: epoch 56 batch 20 loss 0.0014953997451812029
Test: epoch 56 batch 0 loss 0.013074476271867752
epoch 56 finished - avarage train loss 0.007330689659534857  avarage test loss 0.013878089608624578
Training: epoch 57 batch 0 loss 0.008442454040050507
Training: epoch 57 batch 10 loss 0.008689203299582005
Training: epoch 57 batch 20 loss 0.008260846138000488
Test: epoch 57 batch 0 loss 0.015359115786850452
epoch 57 finished - avarage train loss 0.007994303351332402  avarage test loss 0.01717477012425661
Training: epoch 58 batch 0 loss 0.0066860574297606945
Training: epoch 58 batch 10 loss 0.010626566596329212
Training: epoch 58 batch 20 loss 0.004048692528158426
Test: epoch 58 batch 0 loss 0.010818760842084885
epoch 58 finished - avarage train loss 0.012675982434302568  avarage test loss 0.015534335281699896
Training: epoch 59 batch 0 loss 0.0075156730599701405
Training: epoch 59 batch 10 loss 0.008390998467803001
Training: epoc

Training: epoch 85 batch 0 loss 0.003067314624786377
Training: epoch 85 batch 10 loss 0.0016412180848419666
Training: epoch 85 batch 20 loss 0.006302244495600462
Test: epoch 85 batch 0 loss 0.01343118492513895
epoch 85 finished - avarage train loss 0.007219912042712857  avarage test loss 0.013513531535863876
Training: epoch 86 batch 0 loss 0.0022852581460028887
Training: epoch 86 batch 10 loss 0.007489379961043596
Training: epoch 86 batch 20 loss 0.003887064289301634
Test: epoch 86 batch 0 loss 0.01869245059788227
epoch 86 finished - avarage train loss 0.007066436946905893  avarage test loss 0.01924050599336624
Training: epoch 87 batch 0 loss 0.010701825842261314
Training: epoch 87 batch 10 loss 0.009670414961874485
Training: epoch 87 batch 20 loss 0.004877867177128792
Test: epoch 87 batch 0 loss 0.016665775328874588
epoch 87 finished - avarage train loss 0.012751248745440409  avarage test loss 0.017708923434838653
Training: epoch 88 batch 0 loss 0.008153417147696018
Training: epoch 88

Training: epoch 111 batch 20 loss 0.00674485694617033
Test: epoch 111 batch 0 loss 0.011576288379728794
epoch 111 finished - avarage train loss 0.009991497973558205  avarage test loss 0.012920779874548316
Training: epoch 112 batch 0 loss 0.006207141559571028
Training: epoch 112 batch 10 loss 0.0031442337203770876
Training: epoch 112 batch 20 loss 0.0027438243851065636
Test: epoch 112 batch 0 loss 0.013711882755160332
epoch 112 finished - avarage train loss 0.007324010334459358  avarage test loss 0.014969486044719815
Training: epoch 113 batch 0 loss 0.003258145647123456
Training: epoch 113 batch 10 loss 0.004091388080269098
Training: epoch 113 batch 20 loss 0.007794233039021492
Test: epoch 113 batch 0 loss 0.013024095445871353
epoch 113 finished - avarage train loss 0.0077181877728937  avarage test loss 0.01391446846537292
Training: epoch 114 batch 0 loss 0.013074064627289772
Training: epoch 114 batch 10 loss 0.004599754232913256
Training: epoch 114 batch 20 loss 0.007596455980092287
Te

...

Training: epoch 86 batch 0 loss 0.0036913841031491756
Training: epoch 86 batch 10 loss 0.005813811905682087
Training: epoch 86 batch 20 loss 0.0042142379097640514
Test: epoch 86 batch 0 loss 0.01023941207677126
epoch 86 finished - avarage train loss 0.0068736641732012405  avarage test loss 0.014594594715163112
Training: epoch 87 batch 0 loss 0.010221575386822224
Training: epoch 87 batch 10 loss 0.0032170447520911694
Training: epoch 87 batch 20 loss 0.004520980641245842
Test: epoch 87 batch 0 loss 0.009714084677398205
epoch 87 finished - avarage train loss 0.006803804743585402  avarage test loss 0.011894322116859257
Training: epoch 88 batch 0 loss 0.0043801735155284405
Training: epoch 88 batch 10 loss 0.004709809087216854
Training: epoch 88 batch 20 loss 0.008663387969136238
Test: epoch 88 batch 0 loss 0.007894598878920078
epoch 88 finished - avarage train loss 0.007276275379840156  avarage test loss 0.011459972709417343
Training: epoch 89 batch 0 loss 0.005627610720694065
Training: epo

Training: epoch 112 batch 20 loss 0.004779933951795101
Test: epoch 112 batch 0 loss 0.016540100798010826
epoch 112 finished - avarage train loss 0.008242607823219793  avarage test loss 0.016686599468812346
Training: epoch 113 batch 0 loss 0.006017943844199181
Training: epoch 113 batch 10 loss 0.0030161752365529537
Training: epoch 113 batch 20 loss 0.010998585261404514
Test: epoch 113 batch 0 loss 0.015054093673825264
epoch 113 finished - avarage train loss 0.008689161839670148  avarage test loss 0.01571472641080618
Training: epoch 114 batch 0 loss 0.006498516071587801
Training: epoch 114 batch 10 loss 0.014416071586310863
Training: epoch 114 batch 20 loss 0.0061358981765806675
Test: epoch 114 batch 0 loss 0.014088619500398636
epoch 114 finished - avarage train loss 0.007508713992890613  avarage test loss 0.016115690814331174
Training: epoch 115 batch 0 loss 0.003903638571500778
Training: epoch 115 batch 10 loss 0.005883068311959505
Training: epoch 115 batch 20 loss 0.004823003895580769

epoch 139 finished - avarage train loss 0.008138060802593827  avarage test loss 0.015432045562192798
Training: epoch 140 batch 0 loss 0.009916790761053562
Training: epoch 140 batch 10 loss 0.011913455091416836
Training: epoch 140 batch 20 loss 0.004891552031040192
Test: epoch 140 batch 0 loss 0.01492560375481844
epoch 140 finished - avarage train loss 0.009344016193765504  avarage test loss 0.019856350729241967
Training: epoch 141 batch 0 loss 0.013229617848992348
Training: epoch 141 batch 10 loss 0.00857353676110506
Training: epoch 141 batch 20 loss 0.006195798981934786
Test: epoch 141 batch 0 loss 0.011208927258849144
epoch 141 finished - avarage train loss 0.00924449181987037  avarage test loss 0.013468838878907263
Training: epoch 142 batch 0 loss 0.004306560382246971
Training: epoch 142 batch 10 loss 0.0048219566233456135
Training: epoch 142 batch 20 loss 0.006057361140847206
Test: epoch 142 batch 0 loss 0.02136503905057907
epoch 142 finished - avarage train loss 0.0111091757064749

Training: epoch 166 batch 10 loss 0.010730179026722908
Training: epoch 166 batch 20 loss 0.005183397326618433
Test: epoch 166 batch 0 loss 0.010932503268122673
epoch 166 finished - avarage train loss 0.007182298220116003  avarage test loss 0.012636228580959141
Training: epoch 167 batch 0 loss 0.008489005267620087
Training: epoch 167 batch 10 loss 0.003961583133786917
Training: epoch 167 batch 20 loss 0.004140205681324005
Test: epoch 167 batch 0 loss 0.011736867018043995
epoch 167 finished - avarage train loss 0.006569893139510833  avarage test loss 0.014456937555223703
Training: epoch 168 batch 0 loss 0.008877956308424473
Training: epoch 168 batch 10 loss 0.005674684885889292
Training: epoch 168 batch 20 loss 0.0048387497663497925
Test: epoch 168 batch 0 loss 0.010586191900074482
epoch 168 finished - avarage train loss 0.007419825949031731  avarage test loss 0.012674791971221566
Training: epoch 169 batch 0 loss 0.0025188017170876265
Training: epoch 169 batch 10 loss 0.00523221772164106

Test: epoch 192 batch 0 loss 0.030125439167022705
epoch 192 finished - avarage train loss 0.006948837849857478  avarage test loss 0.03711104532703757
Training: epoch 193 batch 0 loss 0.0370161272585392
Training: epoch 193 batch 10 loss 0.08782871067523956
Training: epoch 193 batch 20 loss 0.02633720636367798
Test: epoch 193 batch 0 loss 0.02282845787703991
epoch 193 finished - avarage train loss 0.04981977865099907  avarage test loss 0.023181235417723656
Training: epoch 194 batch 0 loss 0.02370542660355568
Training: epoch 194 batch 10 loss 0.023880837485194206
Training: epoch 194 batch 20 loss 0.018250424414873123
Test: epoch 194 batch 0 loss 0.014257585629820824
epoch 194 finished - avarage train loss 0.01860527938296055  avarage test loss 0.017808564472943544
Training: epoch 195 batch 0 loss 0.009015602059662342
Training: epoch 195 batch 10 loss 0.019328711554408073
Training: epoch 195 batch 20 loss 0.018110932782292366
Test: epoch 195 batch 0 loss 0.016670051962137222
epoch 195 fini

In [31]:
median_loss

0.011459972709417343

In [32]:
losses

[0.011936204391531646,
 0.011919335345737636,
 0.011285196291282773,
 0.012202803627587855,
 0.010931709315627813,
 0.011369480635039508,
 0.011665846221148968,
 0.01126380858477205,
 0.011459972709417343]

In [33]:
df[['x2', 'x1']].head()

,x2,x1
0,-0.199210,-1.242773
1,0.099672,1.030061
2,-1.093608,-0.196601
3,1.788205,1.525318
4,0.431947,0.611544


In [34]:
median_loss_back, losses_back = get_final_median_loss(df[['x2', 'x1']], num_trials)

Training: epoch 0 batch 0 loss 0.5293440222740173
Training: epoch 0 batch 10 loss 0.6268832087516785
Training: epoch 0 batch 20 loss 0.5240335464477539
Test: epoch 0 batch 0 loss 0.4775336980819702
epoch 0 finished - avarage train loss 0.5224528497663038  avarage test loss 0.5293320342898369
Training: epoch 1 batch 0 loss 0.46376171708106995
Training: epoch 1 batch 10 loss 0.45183244347572327
Training: epoch 1 batch 20 loss 0.6972669363021851
Test: epoch 1 batch 0 loss 0.4282788336277008
epoch 1 finished - avarage train loss 0.5176612940327875  avarage test loss 0.5128885433077812
Training: epoch 2 batch 0 loss 0.44308531284332275
Training: epoch 2 batch 10 loss 0.6235655546188354
Training: epoch 2 batch 20 loss 0.52065509557724
Test: epoch 2 batch 0 loss 0.44362887740135193
epoch 2 finished - avarage train loss 0.5215667239550886  avarage test loss 0.5231263786554337
Training: epoch 3 batch 0 loss 0.6598615050315857
Training: epoch 3 batch 10 loss 0.45495253801345825
Training: epoch 3

Training: epoch 28 batch 0 loss 0.005791204050183296
Training: epoch 28 batch 10 loss 0.009249537251889706
Training: epoch 28 batch 20 loss 0.0032975487411022186
Test: epoch 28 batch 0 loss 0.006020783446729183
epoch 28 finished - avarage train loss 0.008175457000796651  avarage test loss 0.018153004115447402
Training: epoch 29 batch 0 loss 0.008491779677569866
Training: epoch 29 batch 10 loss 0.0055030519142746925
Training: epoch 29 batch 20 loss 0.007952672429382801
Test: epoch 29 batch 0 loss 0.0035558755043894053
epoch 29 finished - avarage train loss 0.009325813871390861  avarage test loss 0.01634820323670283
Training: epoch 30 batch 0 loss 0.006720810197293758
Training: epoch 30 batch 10 loss 0.005007367115467787
Training: epoch 30 batch 20 loss 0.004300526808947325
Test: epoch 30 batch 0 loss 0.0034101635683327913
epoch 30 finished - avarage train loss 0.008504314596989545  avarage test loss 0.014756400778423995
Training: epoch 31 batch 0 loss 0.005792582407593727
Training: epoc

Training: epoch 55 batch 0 loss 0.007301707286387682
Training: epoch 55 batch 10 loss 0.010514733381569386
Training: epoch 55 batch 20 loss 0.005645349621772766
Test: epoch 55 batch 0 loss 0.004382119048386812
epoch 55 finished - avarage train loss 0.008697752102181829  avarage test loss 0.013568968395702541
Training: epoch 56 batch 0 loss 0.0049989609979093075
Training: epoch 56 batch 10 loss 0.010814409703016281
Training: epoch 56 batch 20 loss 0.006711605004966259
Test: epoch 56 batch 0 loss 0.007709646597504616
epoch 56 finished - avarage train loss 0.0078086514218614015  avarage test loss 0.01837832317687571
Training: epoch 57 batch 0 loss 0.010523590259253979
Training: epoch 57 batch 10 loss 0.012051045894622803
Training: epoch 57 batch 20 loss 0.011607416905462742
Test: epoch 57 batch 0 loss 0.005980341695249081
epoch 57 finished - avarage train loss 0.011649928200219211  avarage test loss 0.014270613668486476
Training: epoch 58 batch 0 loss 0.007232956122606993
Training: epoch 

Test: epoch 82 batch 0 loss 0.00523136742413044
epoch 82 finished - avarage train loss 0.007996657863259315  avarage test loss 0.014731432893313468
Training: epoch 83 batch 0 loss 0.007097173947840929
Training: epoch 83 batch 10 loss 0.012866683304309845
Training: epoch 83 batch 20 loss 0.00964576005935669
Test: epoch 83 batch 0 loss 0.006633077748119831
epoch 83 finished - avarage train loss 0.008662316956204074  avarage test loss 0.015486346557736397
Training: epoch 84 batch 0 loss 0.011023740284144878
Training: epoch 84 batch 10 loss 0.004733086097985506
Training: epoch 84 batch 20 loss 0.004877698607742786
Test: epoch 84 batch 0 loss 0.005893656052649021
epoch 84 finished - avarage train loss 0.007982889061857915  avarage test loss 0.014180152444168925
Training: epoch 85 batch 0 loss 0.006152545101940632
Training: epoch 85 batch 10 loss 0.00558087881654501
Training: epoch 85 batch 20 loss 0.005233673378825188
Test: epoch 85 batch 0 loss 0.003985796123743057
epoch 85 finished - avar

Training: epoch 109 batch 10 loss 0.003769970964640379
Training: epoch 109 batch 20 loss 0.0054791695438325405
Test: epoch 109 batch 0 loss 0.004962466191500425
epoch 109 finished - avarage train loss 0.009258111344714617  avarage test loss 0.013149886624887586
Training: epoch 110 batch 0 loss 0.006849371828138828
Training: epoch 110 batch 10 loss 0.01022367738187313
Training: epoch 110 batch 20 loss 0.015269486233592033
Test: epoch 110 batch 0 loss 0.003693175036460161
epoch 110 finished - avarage train loss 0.008589323216663867  avarage test loss 0.01262878708075732
Training: epoch 111 batch 0 loss 0.005675601307302713
Training: epoch 111 batch 10 loss 0.010704094544053078
Training: epoch 111 batch 20 loss 0.010515669360756874
Test: epoch 111 batch 0 loss 0.003887514816597104
epoch 111 finished - avarage train loss 0.008873836643158876  avarage test loss 0.012871288519818336
Training: epoch 112 batch 0 loss 0.006301570683717728
Training: epoch 112 batch 10 loss 0.009698281064629555
T

...

Training: epoch 66 batch 20 loss 0.0047094374895095825
Test: epoch 66 batch 0 loss 0.005456568207591772
epoch 66 finished - avarage train loss 0.008531474781318986  avarage test loss 0.01578554220031947
Training: epoch 67 batch 0 loss 0.0072574615478515625
Training: epoch 67 batch 10 loss 0.006632059346884489
Training: epoch 67 batch 20 loss 0.0035963817499578
Test: epoch 67 batch 0 loss 0.003759884973987937
epoch 67 finished - avarage train loss 0.008829908204618198  avarage test loss 0.013477074855472893
Training: epoch 68 batch 0 loss 0.008465928956866264
Training: epoch 68 batch 10 loss 0.0035871509462594986
Training: epoch 68 batch 20 loss 0.005589754786342382
Test: epoch 68 batch 0 loss 0.0035855777096003294
epoch 68 finished - avarage train loss 0.007230104188081519  avarage test loss 0.013207300042267889
Training: epoch 69 batch 0 loss 0.006008070427924395
Training: epoch 69 batch 10 loss 0.007374532055109739
Training: epoch 69 batch 20 loss 0.004262339789420366
Test: epoch 69 

Test: epoch 93 batch 0 loss 0.004211612977087498
epoch 93 finished - avarage train loss 0.00809794503810077  avarage test loss 0.01493831118568778
Training: epoch 94 batch 0 loss 0.004440609365701675
Training: epoch 94 batch 10 loss 0.004221213515847921
Training: epoch 94 batch 20 loss 0.009171826764941216
Test: epoch 94 batch 0 loss 0.00355815258808434
epoch 94 finished - avarage train loss 0.007389359240388048  avarage test loss 0.01294874242739752
Training: epoch 95 batch 0 loss 0.0104090990498662
Training: epoch 95 batch 10 loss 0.005880217999219894
Training: epoch 95 batch 20 loss 0.007129484787583351
Test: epoch 95 batch 0 loss 0.0034484192728996277
epoch 95 finished - avarage train loss 0.0083928019659401  avarage test loss 0.01339356426615268
Training: epoch 96 batch 0 loss 0.004525111522525549
Training: epoch 96 batch 10 loss 0.0048116352409124374
Training: epoch 96 batch 20 loss 0.0038651274517178535
Test: epoch 96 batch 0 loss 0.004357328172773123
epoch 96 finished - avarage

Training: epoch 121 batch 20 loss 0.004034137353301048
Test: epoch 121 batch 0 loss 0.003660363843664527
epoch 121 finished - avarage train loss 0.007456624944662225  avarage test loss 0.014150731905829161
Training: epoch 122 batch 0 loss 0.004290312062948942
Training: epoch 122 batch 10 loss 0.006009768694639206
Training: epoch 122 batch 20 loss 0.0033009429462254047
Test: epoch 122 batch 0 loss 0.004056183155626059
epoch 122 finished - avarage train loss 0.007545000078670424  avarage test loss 0.013512501725926995
Training: epoch 123 batch 0 loss 0.004630516283214092
Training: epoch 123 batch 10 loss 0.004537517204880714
Training: epoch 123 batch 20 loss 0.008987994864583015
Test: epoch 123 batch 0 loss 0.0053518726490437984
epoch 123 finished - avarage train loss 0.00812857347572672  avarage test loss 0.013396454625762999
Training: epoch 124 batch 0 loss 0.004455321468412876
Training: epoch 124 batch 10 loss 0.012679954059422016
Training: epoch 124 batch 20 loss 0.010154807940125465

Training: epoch 149 batch 10 loss 0.009839339181780815
Training: epoch 149 batch 20 loss 0.005077040754258633
Test: epoch 149 batch 0 loss 0.006331935524940491
epoch 149 finished - avarage train loss 0.008605758033307463  avarage test loss 0.01906378217972815
Training: epoch 150 batch 0 loss 0.006779951509088278
Training: epoch 150 batch 10 loss 0.013448633253574371
Training: epoch 150 batch 20 loss 0.00798923708498478
Test: epoch 150 batch 0 loss 0.007148652337491512
epoch 150 finished - avarage train loss 0.007969612849812055  avarage test loss 0.01636843360029161
Training: epoch 151 batch 0 loss 0.006668157875537872
Training: epoch 151 batch 10 loss 0.005374123342335224
Training: epoch 151 batch 20 loss 0.0035156605299562216
Test: epoch 151 batch 0 loss 0.00444988626986742
epoch 151 finished - avarage train loss 0.007968138823093012  avarage test loss 0.015224688919261098
Training: epoch 152 batch 0 loss 0.0073715983889997005
Training: epoch 152 batch 10 loss 0.004351151175796986
Tr

epoch 175 finished - avarage train loss 0.007921013282611966  avarage test loss 0.018960056244395673
Training: epoch 176 batch 0 loss 0.004660699050873518
Training: epoch 176 batch 10 loss 0.0026539620012044907
Training: epoch 176 batch 20 loss 0.007903225719928741
Test: epoch 176 batch 0 loss 0.004182491451501846
epoch 176 finished - avarage train loss 0.007491864167667669  avarage test loss 0.013021119986660779
Training: epoch 177 batch 0 loss 0.003626953111961484
Training: epoch 177 batch 10 loss 0.004662971943616867
Training: epoch 177 batch 20 loss 0.006319697946310043
Test: epoch 177 batch 0 loss 0.0038202910218387842
epoch 177 finished - avarage train loss 0.0075856894415257305  avarage test loss 0.01299387781182304
Training: epoch 178 batch 0 loss 0.0036308274138718843
Training: epoch 178 batch 10 loss 0.008004224859178066
Training: epoch 178 batch 20 loss 0.005256412085145712
Test: epoch 178 batch 0 loss 0.004778990522027016
epoch 178 finished - avarage train loss 0.0068156211

In [35]:
median_loss_back

0.012396272097248584

In [36]:
losses_back

[0.01262878708075732,
 0.012263277196325362,
 0.012966789654456079,
 0.012680116342380643,
 0.012383790162857622,
 0.012234172376338392,
 0.012616099382285029,
 0.012191358720883727,
 0.012396272097248584]

In [ ]:
input_dim = df.shape[1] - 1

train, test = train_test_split(df, test_size=0.1, random_state=10, shuffle=True)

train = np.array(train)
test = np.array(test)

train = MyDataset(train)
test = MyDataset(test)

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, 
                          num_workers=0, pin_memory=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False,
                         num_workers=0, pin_memory=True)

train_loss_avgs, test_loss_avgs, min_loss = train_model(num_epochs)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,8))
ax.plot(train_loss_avgs, label='train')
ax.plot(test_loss_avgs, label='test')
ax.legend()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,8))
ax.plot(train_loss_avgs[100:], label='train')
ax.plot(test_loss_avgs[100:], label='test')
ax.legend()